In [3]:
import pickle
from statistics import mean
import numpy as np
import sys

In [4]:
file_name_train_250 = "RL_trajectories_1000_250.pickle"
file_obj_train_250 = open(file_name_train_250, 'rb')
trajectories_train_250 = pickle.load(file_obj_train_250)
trajectories_train_250 = [list(ele) for ele in trajectories_train_250]

file_name_test_250 = "Human_trajectories_test_250.pickle"
file_obj_test_250 = open(file_name_test_250, 'rb')
trajectories_test_250 = pickle.load(file_obj_test_250)
trajectories_test_250 = [list(ele) for ele in trajectories_test_250]

file_name_train_500 = "RL_trajectories_1000_500.pickle"
file_obj_train_500 = open(file_name_train_500, 'rb')
trajectories_train_500 = pickle.load(file_obj_train_500)
trajectories_train_500 = [list(ele) for ele in trajectories_train_500]

file_name_test_500 = "Human_trajectories_test_500.pickle"
file_obj_test_500 = open(file_name_test_500, 'rb')
trajectories_test_500 = pickle.load(file_obj_test_500)
trajectories_test_500 = [list(ele) for ele in trajectories_test_500]

test_size = 60

In [5]:
def generate_policies(price_low_bound, price_upper_bound, step_size):
    policies = []
    for i in range(price_low_bound, price_upper_bound, step_size):
        for j in range(price_low_bound, price_upper_bound, step_size):
            for k in range(price_low_bound, price_upper_bound, step_size):
                for l in range(price_low_bound, price_upper_bound, step_size):
                    for m in range(4):
                        for n in range(4):
                            for o in range(4):
                                for p in range(3):
                                    policy = (i, j, k, l, m, n, o, p)
                                    policies.append(policy)
    return policies

In [6]:
policies = generate_policies(190, 240, 10)
len(policies)

120000

In [7]:
def evaluate_policy(policy, data, best_buys_cost, best_buys_idx, get_final_output):
    if(len(policy)!=8):
        raise ValueError("Number of parameters in this policy is "+ str(len(policy))+" when it should be 8")
    bought = []
    bought_cost_only = []
    bought_idx_only = []
    t1 = policy[0]
    t2 = policy[1]
    t3 = policy[2]
    t4 = policy[3]
    d1 = policy[4]
    d2 = policy[5]
    d3 = policy[6]
    d4 = policy[7]
    for trajectory in data:
        for idx, cost in enumerate(trajectory):
            local_cat = int(idx/4)
            local_idx = int(idx%4)
            if(idx == len(trajectory) - 1):
                bought.append((cost, idx))
                bought_cost_only.append(cost)
                bought_idx_only.append(idx)
                break
            elif(local_cat == 0):
                if(cost > t1 and local_idx <= d1):
                    continue
                else: 
                    bought.append((cost, idx))
                    bought_cost_only.append(cost)
                    bought_idx_only.append(idx)
                    break
            elif(local_cat == 1):
                if(cost > t2 and local_idx <= d2):
                    continue
                else: 
                    bought.append((cost, idx))
                    bought_cost_only.append(cost)
                    bought_idx_only.append(idx)
                    break
            elif(local_cat == 2):
                if(cost > t3 and local_idx <= d3):
                    continue
                else: 
                    bought.append((cost, idx))
                    bought_cost_only.append(cost)
                    bought_idx_only.append(idx)
                    break
            elif(local_cat == 3):
                if(cost > t4 and local_idx <= d4):
                    continue
                else: 
                    bought.append((cost, idx))
                    bought_cost_only.append(cost)
                    bought_idx_only.append(idx)
                    break
    error = [a_i - b_i for a_i, b_i in zip(bought_cost_only, best_buys_cost)]
    correct_stops = (np.equal(bought_idx_only, best_buys_idx)).astype(int)
    correct_stops = sum(correct_stops)
    avg_error = sum(error)/len(data)
    if(get_final_output):
        return bought, avg_error, correct_stops
    return avg_error, correct_stops

In [8]:
##Cost
best_buy_250_train = [min(ele) for ele in trajectories_train_250]
best_buy_250_test = [min(ele) for ele in trajectories_test_250]
best_buy_500_train = [min(ele) for ele in trajectories_train_500]
best_buy_500_test = [min(ele) for ele in trajectories_test_500]

##Indices
best_buy_250_train_idx = [ele.index(min(ele)) for ele in trajectories_train_250]
best_buy_250_test_idx = [ele.index(min(ele)) for ele in trajectories_test_250]
best_buy_500_train_idx = [ele.index(min(ele)) for ele in trajectories_train_500]
best_buy_500_test_idx = [ele.index(min(ele)) for ele in trajectories_test_500]

In [9]:
def policy_search(price_low_bound, price_upper_bound, data, best_buys_cost, best_buys_idx):
    all_errors = []
    all_correct_stops = []
    all_policies = generate_policies(price_low_bound, price_upper_bound, 10)
    for idx, policy in enumerate(all_policies):
        policy_avg_error, correct_stops = evaluate_policy(policy, data, best_buys_cost, best_buys_idx, False)
        sys.stdout.flush()
        sys.stdout.write("\rPolicies Evaluated: {}/{}".format(idx+1, len(all_policies)))
        all_errors.append(policy_avg_error)
        all_correct_stops.append(correct_stops)
    optimal_policy_by_avg_error = (min(all_errors), all_policies[all_errors.index(min(all_errors))])
    optimal_policy_by_opt_stop = (max(all_correct_stops), all_policies[all_correct_stops.index(max(all_correct_stops))])
    return optimal_policy_by_avg_error, optimal_policy_by_opt_stop



In [22]:
optimal_policy_250 = policy_search(190, 240, trajectories_train_250, best_buy_250_train, best_buy_250_train_idx)
print(optimal_policy_250)
evaluate_policy(optimal_policy_250[0][1], trajectories_test_250, best_buy_250_test, best_buy_250_test_idx)

Policies Evaluated: 120000/120000((10.891, (200, 210, 220, 230, 3, 3, 3, 1)), (577, (200, 200, 210, 210, 3, 3, 3, 1)))


(14.516666666666667, 28)

In [10]:
final_result_avg_250, avg_error, correct_stops = evaluate_policy(optimal_policy_250[0][1], 
                                                                 trajectories_test_250, 
                                                                 best_buy_250_test, 
                                                                 best_buy_250_test_idx, 
                                                                 True)
avg_error, correct_stops

(14.516666666666667, 28)

In [11]:
final_result_opt_stop_250, avg_error, correct_stops = evaluate_policy(optimal_policy_250[1][1], 
                                                                 trajectories_test_250, 
                                                                 best_buy_250_test, 
                                                                 best_buy_250_test_idx, 
                                                                 True)
avg_error, correct_stops

(21.583333333333332, 22)

In [25]:
optimal_policy_500 = policy_search(390, 480, trajectories_train_500, best_buy_500_train, best_buy_500_train_idx)
print("")
print(optimal_policy_500)
evaluate_policy(optimal_policy_500[0][1], trajectories_test_500, best_buy_500_test, best_buy_500_test_idx)

Policies Evaluated: 1578/1259712

KeyboardInterrupt: 

In [ ]:
final_result_avg_500, avg_error, correct_stops = evaluate_policy(optimal_policy_500[0][1], 
                                                                 trajectories_test_500, 
                                                                 best_buy_500_test, 
                                                                 best_buy_500_test_idx, 
                                                                 True)
avg_error, correct_stops

In [ ]:
final_result_opt_stop_500, avg_error, correct_stops = evaluate_policy(optimal_policy_500[1][1], 
                                                                 trajectories_test_500, 
                                                                 best_buy_500_test, 
                                                                 best_buy_500_test_idx, 
                                                                 True)
avg_error, correct_stops

# Final Result

In [ ]:
rl_four_split_avg = [final_result_avg_250, final_result_avg_500]
rl_four_split_opt_stop = [final_result_opt_stop_250, final_result_opt_stop_500]

In [2]:
def save_data(obj, filename):
    final_file_object = open(filename, 'wb')
    pickle.dump(obj, final_file_object)

In [ ]:
save_data(rl_four_split_avg, "rl_four_split_avg")
save_data(rl_four_split_opt_stop, "rl_four_split_opt_stop")